In [1]:
access_token = '0df761baebc99c1558c737e3f9d6d20df2a50cc5bc55df89b3549b413397c254de464354b09da7837a725'
version = '5.130'

In [2]:
import requests
import json
import time
from tqdm.notebook import tqdm
from datetime import datetime

In [3]:
def dictToListOfStrings(dictionary):
    listOfStrings=[]
    for key in dictionary:
        listOfStrings.append(str(key) + '=' + str(dictionary[key]))
    return listOfStrings

def vkMethod(method_name, parameters, access_token, version):
    url = 'https://api.vk.com/method/{method_name}?{parameters}&access_token={access_token}&v={version}'
    url = url.format(method_name=method_name,
               parameters='&'.join(dictToListOfStrings(parameters)),
               access_token=access_token,
               version=version)
    time.sleep(0.3)
    response = requests.get(url).json()
    return response

def createBatches(someList, batchSize=25):
    batches = []
    for i in range(0, len(someList), batchSize):
        batches.append(someList[i: i + batchSize])
    return batches

def createBatchedExecuteCode(method, params, batchSize, totalCount = None):
    apiRequests = []
    totalCount = getObjectCount(method, params) if totalCount is None else totalCount
    for i in range(0, totalCount, 100):
        params_i = params.copy()
        params_i['offset'] = i
        request = 'API.{method}({params})'.format(method = method, params = params_i)
        apiRequests.append(request)
    return createBatches(apiRequests, batchSize)

def getObjectCount(method, params):
    response = vkMethod(method, params, access_token, version)
    return response['response']['count']

fieldsToKeep = ['id', 'owner_id', 'date', 'text', 'marked_as_ads', 'comments', 'likes', 'reposts', 'views', 'thread', 'post_id']
def deleteFromDict(dictionary, fieldsToKeep):
    new = dictionary.copy()
    for field in list(new.keys() - fieldsToKeep):
        new.pop(field, None)
    return new

In [4]:
import json
with open('config.json', 'r') as f:
    properties = json.loads(f.read())

communities_params = properties['communities_params']
filters = properties['filters']

In [5]:
date_time_obj = datetime.strptime('2019-11-01', '%Y-%m-%d')
first_news = int(time.mktime(date_time_obj.timetuple()))


def rawPosts(community_params):
    allWalls = []
    params = {'owner_id': community_params['community_id'], 'count':100}
    codeInBatches = createBatchedExecuteCode('wall.get', params, batchSize = 12)
    for code in tqdm(codeInBatches):
        executeRequests = ', '.join(code)
        executeCode = 'return[{requests}];'.format(requests = executeRequests)
        response = vkMethod('execute', {'code': executeCode}, access_token, version)['response']
        
        for batch in response:
            allWalls += batch['items']
        
        if allWalls[-1]['date'] < first_news:
            break
    with open('posts/rawPosts_{name}.json'.format(name=community_params['name']), 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allWalls))
        if community_params['tag_filter']:
            clean = list(
                filter(
                    lambda item: any(filter_ in item['text'].lower() for filter_ in filters),
                    clean
                )
            )
        json.dump(clean, f)
    return len(codeInBatches)

In [ ]:
rawPosts(communities_params[0])
rawPosts(communities_params[1])
rawPosts(communities_params[2])
rawPosts(communities_params[3])
rawPosts(communities_params[4])
rawPosts(communities_params[5])
rawPosts(communities_params[6])
rawPosts(communities_params[7])
rawPosts(communities_params[8])

In [10]:
def getPostIdsAndCounts(community_params):
    ids = []
    with open('posts/rawPosts_{name}.json'.format(name=community_params['name'])) as json_file:
        data = json.load(json_file)
        for post in data:
            ids.append((post['id'], post['comments']['count']))
    return ids

In [11]:
def rawComments(community_params):
    allComments = []
    post_ids = getPostIdsAndCounts(community_params)
    for post_id, count in tqdm(post_ids):
        params = {'owner_id': community_params['community_id'], 'post_id':post_id, 'count':100}
        codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 25, totalCount=count)
        for code in codeInBatches:
            executeRequests = ', '.join(code)
            executeCode = 'return[{requests}];'.format(requests = executeRequests)
            response = vkMethod('execute', {'code': executeCode}, access_token, version)
            try:
                response = response['response']
            except:
                print(response)
            for batch in response:
                comments = batch['items'].copy()
                for comment in comments:
                    comment['post_id'] = post_id
                allComments += comments
    with open('comments/rawComments_{name}.json'.format(name=community_params['name']), 'w+') as f:
        json.dump(allComments, f)
    return len(codeInBatches)

In [ ]:
for community_params in communities_params:
    rawComments(community_params)

In [16]:
def extractCommentIdsAndPostIdAndCount(community_params):
    ids = []
    with open('comments/rawComments_{name}.json'.format(name=community_params['name'])) as json_file:
        data = json.load(json_file)
        for comment in data:
            ids.append((comment['id'], comment['post_id'], comment['thread']['count']))
    return ids

def getAnswerTexts(community_params):
    allComments = []
    for commentId, postId, count in tqdm(extractCommentIdsAndPostIdAndCount(community_params)):
        if count > 0:
            params = {'owner_id': community_params['community_id'], 'post_id':postId, 'comment_id':commentId, 'count':100}
            codeInBatches = createBatchedExecuteCode('wall.getComments', params, batchSize = 25, totalCount = count)
            for code in codeInBatches:
                executeRequests = ', '.join(code)
                executeCode = 'return[{requests}];'.format(requests = executeRequests)
                response = vkMethod('execute', {'code': executeCode}, access_token, version)
                try:
                    response = response['response']
                except:
                    print(response)
                for batch in response:
                    allComments += batch['items']
    with open('answers/rawAnswers_{name}.json'.format(name=community_params['name']), 'w+') as f:
        clean = list(map(lambda item: deleteFromDict(item, fieldsToKeep), allComments))
        json.dump(clean, f)

In [ ]:
getAnswerTexts(communities_params[0])
getAnswerTexts(communities_params[1])
getAnswerTexts(communities_params[2])
getAnswerTexts(communities_params[3])
getAnswerTexts(communities_params[4])
getAnswerTexts(communities_params[5])
getAnswerTexts(communities_params[6])
getAnswerTexts(communities_params[7])
getAnswerTexts(communities_params[8])